In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import holidays
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import optuna
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
import xgboost as xgb
from statsmodels.tsa.filters.hp_filter import hpfilter

In [ ]:
data = pd.read_csv("train_temporary_dtw.csv")

In [ ]:
data

### 파생변수 생성
1. 평일,주말
2. 공휴일

In [ ]:
# 요일 추출 (0=월요일, 6=일요일)
data['TM'] = pd.to_datetime(data['TM'], format='%Y%m%d%H')
data['year'] = data['TM'].dt.year
data['month'] = data['TM'].dt.month
data['day'] = data['TM'].dt.day
data['hour'] = data['TM'].dt.hour

data['weekday'] = data['TM'].dt.weekday

# 평일/주말 구분 (0=평일, 1=주말)
data['is_weekend'] = (data['weekday'] >= 5).astype(int)

# 한국 공휴일 설정
korea_holidays = holidays.Korea(years=range(2021, 2025))

# 공휴일 여부 (1=공휴일, 0=평일)
data['is_holiday'] = data['TM'].dt.date.apply(lambda x: x in korea_holidays).astype(int)

data.drop("weekday",axis=1, inplace=True)
data.head()

In [ ]:
def add_fourier_features(df, col, period, order=3, prefix=None):
    """
    Append 2*order Fourier terms for a cyclical column.

    Parameters
    ----------
    df      : pd.DataFrame              (original data)
    col     : str                       (column holding integers 0-(P-1) or 1-P)
    period  : int                       (cycle length, e.g. 24 for hour, 12 for month)
    order   : int, default 3            (# harmonics K)
    prefix  : str or None               (prefix for new columns; defaults to `col`)
    """
    prefix = prefix or col
    out = df.copy()
    x = out[col].astype(float)

    for k in range(1, order + 1):
        angle = 2.0 * np.pi * k * x / period
        out[f"{prefix}_sin{k}"] = np.sin(angle)
        out[f"{prefix}_cos{k}"] = np.cos(angle)
    return out

# Hour
data = add_fourier_features(data, col='hour', period=24, order=2, prefix='hour')

# Month
data = add_fourier_features(data, col='month', period=12, order=2, prefix='month')



data['days_in_month'] = data['TM'].dt.days_in_month

def fourier_dom(df, order=3, day_col='day', period_col='days_in_month', prefix='dom'):
    out = df.copy()
    d   = out[day_col].astype(float)
    P   = out[period_col].astype(float)
    for k in range(1, order + 1):
        angle = 2 * np.pi * k * d / P
        out[f'{prefix}_sin{k}'] = np.sin(angle)
        out[f'{prefix}_cos{k}'] = np.cos(angle)
    return out

data = fourier_dom(data, order=2,prefix='day')

3. 불쾌지수(DI)  
DI = 0.4×(Ta + Tw) + 15  → 일반적인 식  
Ta : 건구온도  
Tw : 습구온도 (없을 시 밑의 식으로)  

DI = 9/5×Ta - 0.55×(1 - RH)×(9/5×Ta - 26) + 32  
RH : 상대습도 (소수 단위)

In [ ]:
def calculate_DI(ta, hm):
    hm = hm / 100
    DI = 1.8*ta - 0.55*(1 - hm)*(1.8*ta - 26) +32
    return DI

In [ ]:
def DI_level(di):
    if di < 68:
        return 0  # 전원 쾌적
    elif di < 70:
        return 1  # 불쾌감 나타남  
    elif di < 75:
        return 2  # 10% 정도 불쾌
    elif di < 80:
        return 3  # 50% 정도 불쾌
    elif di < 83:
        return 4  # 전원 불쾌
    else:
        return 5  # 매우 불쾌

In [ ]:
data['DI'] = calculate_DI(data['TA'], data['HM'])
data['Discomfort_level'] = data['DI'].apply(DI_level)

4. 난방도일 (HDD) 

원래는 일별로 구하지만 그렇게 하면 같은 값이 24개씩 반복되기 때문에 시간별 난방도일로 계산

In [ ]:
def create_HDD_features(
    data: pd.DataFrame,
    *,
    branch_col: str = "branch_ID",   # 그룹 구분 열
    temp_col: str   = "TA",          # 시간별 실외 온도
    time_col: str   = "TM",          # 타임스탬프
    base_temp: int  = 18,            # 기준 온도(°C)
    windows: tuple  = (7, 30)        # 누적 일수(예: 7일·30일)
) -> pd.DataFrame:
    df = data.copy()

    df = df.sort_values([branch_col, time_col])

    # 3) 시간별 HDH = (base – TA)^+    (음수면 0으로 잘림)
    df["hourly_HDH"] = (base_temp - df[temp_col]).clip(lower=0)

    # 4) 최근 N일 누적 HDD — 그룹별 rolling 합계
    for days in windows:
        hours = 24 * days                        # 창 크기(시간)
        df[f"rolling_HDD_{days}d"] = (
            df
            .groupby(branch_col, observed=True)["hourly_HDH"]
            .transform(lambda s: s.rolling(window=hours, min_periods=1).sum())
        )

    return df

# 사용
data = create_HDD_features(
    data,            # 원본 DataFrame
    branch_col="branch_ID",
    temp_col="TA",
    time_col="TM",
    base_temp=18,
    windows=(7, 30)  # 7일 및 30일 누적 HDD
)

### HP 필터

In [ ]:
# 2) 그룹별 HP 필터 함수
def hp_filter_group(group, column="heat_demand", lam=1600):
    """
    한 그룹(branch_ID)에 대해 HP 필터 적용 후 trend·noise 열을 추가해 돌려줌
    """
    noise, trend     = hpfilter(group[column], lamb=lam)
    group            = group.copy()          # 원본 손상 방지
    group["heat_demand_trend"]   = trend
    group["heat_demand_cycle"]   = noise
    return group

# 3) 정렬(시간 순서) → 그룹별 apply → 결과를 하나로
data = (
    data.sort_values(["branch_ID", "TM"])
      .groupby("branch_ID", group_keys=False)
      .apply(hp_filter_group)   # <- 추가
)

In [ ]:
# lag features 생성
def create_lag_features(df, column, lags):
    """
    Create lag features for a specific column in the DataFrame.
    
    Parameters:
    df (DataFrame): Input DataFrame.
    column (str): Column name to create lag features for.
    lags (list): List of lag periods to create features for.
    
    Returns:
    DataFrame: DataFrame with lag features added.
    """
    for lag in lags:
        df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    return df

lag_dict = {'TA':[1,3,6,12,24], 'HM':[1,2,3], 'WS':[1,3],'SI':[1]}

# grouped by 'branch_ID'

for col, lags in lag_dict.items():
    for lag in lags:
        data[f"{col}_lag_{lag}"] = (
            data
            .groupby("branch_ID")[col]          # 그룹별 시계열
            .transform(lambda s: s.shift(lag))  # 그룹 내부에서만 shift
        )

In [ ]:
# 타겟 변수와 피처 분리
target_col = 'heat_demand_cycle'
exclude_cols = ['TM', target_col, 'year','month', 'day','hour','Unnamed: 0','branch_ID',"WD","RN_DAY",'days_in_month', 'heat_demand_trend','heat_demand']
feature_cols = [col for col in data.columns if col not in exclude_cols]

In [ ]:
feature_cols

In [ ]:
data.dropna(subset=feature_cols,inplace=True)
len(data)

In [ ]:
data['branch_ID'] = data['branch_ID'].astype('category')

print(f"\nTarget 변수: {target_col}")
print(f"사용될 Feature 개수: {len(feature_cols)}")

In [ ]:
# 연도 컬럼 추가
data['year'] = pd.to_datetime(data['TM']).dt.year

# 바로 분할
train_data = data[data['year'].isin([2021, 2022])]
val_data = data[data['year'] == 2023]

Branch ID별 모델 학습

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna
import joblib

def find_best_params_by_branch(data: pd.DataFrame, feature_cols: list, target_col: str) -> dict:
    best_params_dict = {}
    unique_branches = data['branch_ID'].unique()
    
    # 학습용과 검증용 데이터 분리
    train_data = data[data['year'].isin([2021, 2022])].copy()
    val_data = data[data['year'] == 2023].copy()
    
    for branch_id in unique_branches:
        print(f"\n===== Finding Best Parameters for branch_ID: {branch_id} =====")
        
        # 지사별 데이터 필터링
        branch_train = train_data[train_data['branch_ID'] == branch_id].copy()
        branch_val = val_data[val_data['branch_ID'] == branch_id].copy()
        
        # 시계열 순서대로 정렬
        branch_train = branch_train.sort_values('TM').reset_index(drop=True)
        branch_val = branch_val.sort_values('TM').reset_index(drop=True)
        
        X_train = branch_train[feature_cols]
        y_train = branch_train[target_col]
        X_val = branch_val[feature_cols]
        y_val = branch_val[target_col]
        
        # 시계열 교차검증 설정 (학습 데이터 내에서)
        tscv = TimeSeriesSplit(n_splits=5)
        
        # Optuna objective 함수 (CV 적용)
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 500, 5000, step=100),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.7, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.7, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
                'gamma': trial.suggest_float('gamma', 1e-8, 0.5, log=True),
                'random_state': 42,
                'n_jobs': -1,
                'verbosity': 0
            }
            
            # CV 점수들을 저장할 리스트
            cv_scores = []
            
            # TimeSeriesSplit을 실제로 사용하여 CV 수행
            for train_idx, val_idx in tscv.split(X_train):
                X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                
                # 모델 생성 및 학습
                xgb_model = xgb.XGBRegressor(**params)
                xgb_model.fit(X_fold_train, y_fold_train)
                
                # fold별 예측 및 RMSE 계산
                y_pred = xgb_model.predict(X_fold_val)
                rmse = np.sqrt(mean_squared_error(y_fold_val, y_pred))
                cv_scores.append(rmse)
            
            # CV 점수들의 평균을 반환 (최적화 목표)
            return np.mean(cv_scores)
        
        print(f"--- Starting Optuna Hyperparameter Tuning for {branch_id} ---")
        
        # Optuna study 생성 및 최적화
        study = optuna.create_study(direction='minimize', 
                                  sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=50)
        
        # 최적 파라미터로 2023 검증 데이터 평가
        best_params = study.best_params.copy()
        best_params.update({
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0
        })
        
        final_model = xgb.XGBRegressor(**best_params)
        final_model.fit(X_train, y_train)  # 2021-2022 전체로 학습
        y_val_pred = final_model.predict(X_val)  # 2023으로 예측
        val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        
        best_params_dict[branch_id] = study.best_params
        print(f"--- Best Parameters for {branch_id}: {study.best_params} ---")
        print(f"--- Best CV RMSE for {branch_id}: {study.best_value:.4f} ---")
        print(f"--- 2023 Validation RMSE for {branch_id}: {val_rmse:.4f} ---")
    
    return best_params_dict

def train_final_models_with_best_params(data: pd.DataFrame, feature_cols: list, target_col: str, best_params_dict: dict) -> dict:
    final_models = {}
    unique_branches = data['branch_ID'].unique()
    
    # 2021-2023 전체 데이터 (최종 학습용)
    full_train_data = data[data['year'].isin([2021, 2022, 2023])].copy()
    
    for branch_id in unique_branches:
        print(f"\n===== Training Final Model for branch_ID: {branch_id} =====")
        
        # 지사별 데이터 필터링 및 정렬
        branch_data = full_train_data[full_train_data['branch_ID'] == branch_id].copy()
        branch_data = branch_data.sort_values('TM').reset_index(drop=True)
        
        X = branch_data[feature_cols]
        y = branch_data[target_col]
        
        # 해당 지사의 최적 파라미터 가져오기
        best_params = best_params_dict[branch_id].copy()
        best_params.update({
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0
        })
        
        # 최종 모델 학습
        final_model = xgb.XGBRegressor(**best_params)
        final_model.fit(X, y)
        
        final_models[branch_id] = final_model
        print(f"--- Final model training completed for {branch_id} ---")
    
    return final_models

def predict_with_final_models(data_pred: pd.DataFrame, models: dict, feature_cols: list) -> pd.DataFrame:
    predictions = []
    
    data_pred_sorted = data_pred.sort_values('branch_ID').copy()
    
    for branch_id, group_data in data_pred_sorted.groupby('branch_ID'):
        print(f"--- Predicting for branch_ID: {branch_id} ---")
        
        if branch_id in models:
            model = models[branch_id]
            X_pred = group_data[feature_cols]
            group_predictions = model.predict(X_pred)
            result_data = group_data.copy()
            result_data['prediction'] = group_predictions
            predictions.append(result_data)
    
    final_data = pd.concat(predictions).sort_index()
    return final_data

In [ ]:
# 1단계: 2021-2022 학습, 2023 검증으로 최적 파라미터 찾기
print("=" * 60)
print("STEP 1: Finding Best Parameters using 2021-2022 train, 2023 validation")
print("=" * 60)

best_params_dict = find_best_params_by_branch(data, feature_cols, target_col)

# 최적 파라미터 저장
joblib.dump(best_params_dict, 'best_params_by_branch_modify1_cv.joblib')
print(f"\nBest parameters saved for {len(best_params_dict)} branches")

final_models = train_final_models_with_best_params(
    data         = train_data,  # <─ 2023 제외
    feature_cols = feature_cols,
    target_col   = target_col,
    best_params_dict = best_params_dict
)

# 3) 2023 데이터 예측
pred_2023 = predict_with_final_models(
    data_pred    = val_data,  # <─ 2023만 전달
    models       = final_models,
    feature_cols = feature_cols
)
pred_2023 = pred_2023.to_csv('2023_cycle_lag.csv', index=False)

# 2단계: 최적 파라미터로 2021-2023 전체 데이터 재학습
print("\n" + "=" * 60)
print("STEP 2: Training Final Models with 2021-2023 data using best parameters")
print("=" * 60)

final_models = train_final_models_with_best_params(data, feature_cols, target_col, best_params_dict)



# 최종 모델 저장
joblib.dump(final_models, 'final_models_2021_2023_modify1_cv.joblib')
print(f"\nFinal models saved for {len(final_models)} branches")

In [ ]:
cycle = pd.read_csv('2023_cycle_lag.csv')
trend = pd.read_csv("2023_trend_lag.csv")

pred = cycle['prediction'].values + trend['prediction'].values
# pred = np.round(pred, 1)
real = val_data['heat_demand'].values
# real = np.round(val_data['heat_demand'].values)
rmse = np.sqrt(mean_squared_error(real, pred))

print(rmse)

In [ ]:
import matplotlib.pyplot as plt

def plot_heat_demand(
    df: pd.DataFrame,
    trend: pd.DataFrame,
    cycle: pd.DataFrame,
    region: str,
    lw: float = 1.0,
    show_obs_sum: bool = False,   # ← 실측 합 라인 표시 여부
) -> None:
    """
    지역별 Heat-Demand · Trend · Cycle   (실측 ↔ 예측) + 예측 합 비교.

    Parameters
    ----------
    df, trend, cycle : 이전과 동일
    region           : branch_ID
    lw               : 선 굵기
    show_obs_sum     : True 면 (trend+cycle) 실측 합도 표시
    """

    # ── 1. TM 타입 통일
    for _df in (df, trend, cycle):
        _df['TM'] = pd.to_datetime(_df['TM'])

    # ── 2. 필터링 · 병합
    region_df = df.query("branch_ID == @region").copy()
    if region_df.empty:
        raise ValueError(f"'{region}' 지역 데이터가 없습니다.")

    trend_pred = (
        trend.query("branch_ID == @region")[['TM', 'prediction']]
        .rename(columns={'prediction': 'trend_pred'})
    )
    cycle_pred = (
        cycle.query("branch_ID == @region")[['TM', 'prediction']]
        .rename(columns={'prediction': 'cycle_pred'})
    )

    plot_df = (
        region_df
        .merge(trend_pred, on='TM', how='left')
        .merge(cycle_pred, on='TM', how='left')
        .sort_values('TM')
    )

    # ── 3. 합계 계산
    plot_df['pred_sum'] = plot_df['trend_pred'] + plot_df['cycle_pred']
    if show_obs_sum:
        plot_df['obs_sum'] = (
            plot_df['heat_demand_trend'] + plot_df['heat_demand_cycle']
        )

    # ── 4. 플롯
    fig, axes = plt.subplots(3, 1, figsize=(14, 9), sharex=True,
                             gridspec_kw={'hspace': 0.35})

    # (a) Heat Demand: 실측 vs 예측 합
    axes[0].plot(
        plot_df['TM'], plot_df['heat_demand'],
        label='Observed', color='tab:blue', linewidth=lw
    )
    axes[0].plot(
        plot_df['TM'], plot_df['pred_sum'],
        label='Predicted (trend+cycle)', color='tab:red',
        linewidth=lw
    )
    if show_obs_sum:
        axes[0].plot(
            plot_df['TM'], plot_df['obs_sum'],
            label='Observed (trend+cycle)', color='tab:green',
            linewidth=lw, linestyle=':'
        )
    axes[0].set(title=f'Heat Demand – Region {region}')

    # (b) Trend: 실측 vs 예측
    axes[1].plot(
        plot_df['TM'], plot_df['heat_demand_trend'],
        label='Observed', color='tab:blue', linewidth=lw
    )
    axes[1].plot(
        plot_df['TM'], plot_df['trend_pred'],
        label='Predicted', color='tab:orange', linewidth=lw
    )
    axes[1].set(title='Trend Component')

    # (c) Cycle: 실측 vs 예측
    axes[2].plot(
        plot_df['TM'], plot_df['heat_demand_cycle'],
        label='Observed', color='tab:blue', linewidth=lw
    )
    axes[2].plot(
        plot_df['TM'], plot_df['cycle_pred'],
        label='Predicted', color='gold', linewidth=lw
    )
    axes[2].set(title='Noise Component')

    # ── 공통 옵션
    for ax in axes:
        ax.grid(alpha=0.3)
        ax.legend(loc='upper left')

    fig.autofmt_xdate()   # 날짜 라벨 자동 회전
    plt.show()

plot_heat_demand(val_data,trend,cycle,'A')

### test data 파생변수 처리

In [ ]:
data1 = pd.read_csv("train_temporary_dtw.csv")
data2 = pd.read_csv('test_temporary_dtw.csv')
datafull = pd.concat([data1,data2])

In [ ]:
data_test = datafull
# 요일 추출 (0=월요일, 6=일요일)
data_test['TM'] = pd.to_datetime(data_test['TM'], format='%Y%m%d%H')
data_test['year'] = data_test['TM'].dt.year
data_test['month'] = data_test['TM'].dt.month
data_test['day'] = data_test['TM'].dt.day
data_test['hour'] = data_test['TM'].dt.hour

data_test['weekday'] = data_test['TM'].dt.weekday

# 평일/주말 구분 (0=평일, 1=주말)
data_test['is_weekend'] = (data_test['weekday'] >= 5).astype(int)

# 한국 공휴일 설정
korea_holidays = holidays.Korea(years=range(2021, 2025))

# 공휴일 여부 (1=공휴일, 0=평일)
data_test['is_holiday'] = data_test['TM'].dt.date.apply(lambda x: x in korea_holidays).astype(int)

data_test.drop("weekday",axis=1, inplace=True)
data_test.head()

In [ ]:
#Hour
data_test = add_fourier_features(data_test, col='hour', period=24, order=2, prefix='hour')

# Month
data_test = add_fourier_features(data_test, col='month', period=12, order=2, prefix='month')

data_test['days_in_month'] = data_test['TM'].dt.days_in_month

data_test = fourier_dom(data_test, order=2, prefix='day')

In [ ]:
data_test['DI'] = calculate_DI(data_test['TA'], data_test['HM'])
data_test['Discomfort_level'] = data_test['DI'].apply(DI_level)

In [ ]:
data_test = create_HDD_features(
    data_test,            # 원본 DataFrame
    branch_col="branch_ID",
    temp_col="TA",
    time_col="TM",
    base_temp=18,
    windows=(7, 30)  # 7일 및 30일 누적 HDD
)

In [ ]:
# grouped by 'branch_ID'
for col, lags in lag_dict.items():
    for lag in lags:
        data_test[f"{col}_lag_{lag}"] = (
            data_test
            .groupby("branch_ID")[col]          # 그룹별 시계열
            .transform(lambda s: s.shift(lag))  # 그룹 내부에서만 shift
        )

In [ ]:
data_test

In [ ]:
data_test = data_test[data_test['year'].isin([2024,2025])]
data_test

In [ ]:
# 타겟 변수와 피처 분리
target_col = 'heat_demand_cycle'
exclude_cols = ['TM', target_col, 'year','month', 'day','hour','Unnamed: 0','branch_ID',"WD","RN_DAY",'days_in_month', 'heat_demand_trend','heat_demand']
feature_cols = [col for col in data.columns if col not in exclude_cols]

In [ ]:
data_test['branch_ID'] = data_test['branch_ID'].astype('category')

print(f"\nTarget 변수: {target_col}")
print(f"사용될 Feature 개수: {len(feature_cols)}")

In [ ]:
# 3단계: 2024년 데이터로 최종 예측
print("\n" + "=" * 60)
print("STEP 3: Making final predictions on 2024 test data")
print("=" * 60)

# 2024년 테스트 데이터 준비 (이미 전처리된 data_test 사용)
test_data_2024 = data_test.copy()
test_data_for_pred = test_data_2024.drop(columns=[target_col], errors='ignore')

# 최종 예측 수행
final_predictions = predict_with_final_models(test_data_for_pred, final_models, feature_cols)

# 예측 결과 저장
prediction_results = np.round(final_predictions['prediction'], 1)
pred_df = pd.DataFrame({'pred': final_predictions['prediction']})
pred_df.to_csv('2024_pred_cycle.csv', index=False)

In [ ]:
cycle = pd.read_csv('2024_pred_cycle.csv')
trend = pd.read_csv('2024_pred_trend.csv')
pred = cycle['pred'].values + trend['pred'].values
pred = np.round(pred, 1)
pred_df = pd.DataFrame({'prediction': pred})
pred_df.to_csv('2024_final_prediction.csv', index=False)